# Load in R

In [1]:
# Essential packages
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import muon as mu  # specifically for multiome data
import matplotlib.pyplot as plt

# For reading R objects
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# For reading 10x HDF5 files
import h5py
import os

In [2]:
import rpy2
pandas2ri.activate()
%load_ext rpy2.ipython

In [3]:
wd_dir = '/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_SET/Multiome_SETBP1'
os.chdir(wd_dir)

In [4]:
%%R
load("/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_SET/Multiome_SETBP1/Multiome_SETBP1/all/AGG_Setbp1/outs/Multiome.RData")
ls()  #list all objects that were loaded


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
     [1] "A"                   "A2"                  "all.genes"          
 [4] "Cluster"             "cluster.markers"     "clusters_coupe"     
 [7] "DGE_embryo_ctrl_mut" "DGE_P2_ctrl_mut"     "i"                  
[10] "LogFC.onlypos"       "min.diff.pct"        "min.pct"            
[13] "mycol"               "N"                   "p1"                 
[16] "p2"                  "p3"                  "pbmc_RNA"           
[19] "pbmc_RNA_prova"      "pbmc_umap_coord"     "PCA"                
[22] "test.use"            "thresh.use"          "top10"              


In [14]:
%%R
pbmc_RNA

An object of class Seurat 
24993 features across 34264 samples within 1 assay 
Active assay: RNA (24993 features, 0 variable features)
 2 layers present: counts, data


In [5]:
%%R 
# Load required libraries
library(Seurat)
library(ggplot2)
library(dplyr)
library(pheatmap)

Loading required package: SeuratObject
Loading required package: sp

Attaching package: ‘SeuratObject’

The following objects are masked from ‘package:base’:

    intersect, t


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

In addition: Warning message:
package ‘pheatmap’ was built under R version 4.4.1 


In [15]:
%%R
# Basic information about RNA object
print("RNA object information:")
pbmc_RNA

[1] "RNA object information:"
An object of class Seurat 
24993 features across 34264 samples within 1 assay 
Active assay: RNA (24993 features, 0 variable features)
 2 layers present: counts, data


In [16]:
%%R
print("Available reductions:")
print(Reductions(pbmc_RNA))

[1] "Available reductions:"
NULL


In [22]:
%%R
# For embryo samples
head(DGE_embryo_ctrl_mut)

               p_val  avg_diff pct.1 pct.2     p_val_adj Gene_name
Xist    0.000000e+00 -4.549018 0.227 0.870  0.000000e+00      Xist
Tsix   9.366323e-280 -1.184269 0.166 0.636 2.340925e-275      Tsix
Hbb-bs 5.653742e-242  2.375886 0.820 0.387 1.413040e-237    Hbb-bs
Tmsb10 2.876233e-216  3.127081 0.940 0.691 7.188570e-212    Tmsb10
Rps21  3.197631e-206  2.463274 0.903 0.643 7.991840e-202     Rps21
Rpl37  3.932163e-191  2.409653 0.906 0.678 9.827655e-187     Rpl37


In [24]:
%%R
# For P2 samples
head(DGE_P2_ctrl_mut)

         p_val  avg_diff pct.1 pct.2 p_val_adj Gene_name
Mir99ahg     0  9.475910 0.986 0.964         0  Mir99ahg
Grik2        0  4.446774 0.905 0.868         0     Grik2
Hba-a1       0 -1.172631 0.396 0.652         0    Hba-a1
Hbb-bs       0 -1.306164 0.568 0.815         0    Hbb-bs
Dbi          0 -1.767954 0.830 0.954         0       Dbi
Cmss1        0 -4.744029 0.802 0.935         0     Cmss1


In [26]:
%%R
# Check cluster markers
head(cluster.markers)

[1] "Top markers per cluster:"
        p_val avg_logFC pct.1 pct.2 p_val_adj cluster    gene
Unc5d       0  1.748728 0.961 0.489         0       0   Unc5d
Sema3c      0  1.571726 0.845 0.230         0       0  Sema3c
Galnt17     0  1.350758 0.890 0.348         0       0 Galnt17
Gramd1b     0  1.334421 0.902 0.421         0       0 Gramd1b
Frmd4b      0  1.314171 0.967 0.612         0       0  Frmd4b
Zfhx4       0  1.276921 0.837 0.365         0       0   Zfhx4


In [ ]:
%%R
# ggsave("umap_clusters.pdf", p1, width = 10, height = 8)
# ggsave("volcano_plot.pdf", p2, width = 10, height = 8)
# ggsave("feature_plots.pdf", p3, width = 15, height = 10)

# Load in Python


In [ ]:
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

In [48]:
def load_rdata(rdata_path):
    """
    Load an RData file and convert it to Python objects
    """
    # Initialize R
    r = robjects.r
    
    # Load the RData file
    r['load'](rdata_path)
    
    # Get the names of objects in the R workspace
    r_objects = list(r['ls']())
    
    # Initialize dictionary to store converted objects
    converted_objects = {}
    
    # Convert each R object to Python
    with localconverter(robjects.default_converter + pandas2ri.converter):
        for obj_name in r_objects:
            r_obj = r[obj_name]
            
            try:
                # If object is already a pandas DataFrame or numpy array, store directly
                if isinstance(r_obj, (pd.DataFrame, np.ndarray)):
                    converted_objects[obj_name] = r_obj
                    continue
                
                # Handle Seurat objects specifically
                if isinstance(r_obj, robjects.methods.RS4):
                    converted_objects[obj_name] = r_obj
                    print(f"Stored Seurat object {obj_name} in original R format")
                    continue
                
                # Handle different R object types
                if isinstance(r_obj, (robjects.vectors.DataFrame, robjects.vectors.Matrix)):
                    converted_objects[obj_name] = pandas2ri.rpy2py(r_obj)
                elif isinstance(r_obj, robjects.vectors.Array):
                    converted_objects[obj_name] = np.array(r_obj)
                elif isinstance(r_obj, robjects.vectors.StrVector):
                    converted_objects[obj_name] = list(r_obj)
                elif isinstance(r_obj, robjects.vectors.FloatVector):
                    converted_objects[obj_name] = np.array(r_obj)
                elif isinstance(r_obj, robjects.vectors.BoolVector):
                    converted_objects[obj_name] = np.array(r_obj, dtype=bool)
                elif str(type(r_obj)).find('OrdDict') != -1:  # Check if it's an OrdDict without direct import
                    try:
                        # Convert OrdDict to regular dictionary
                        converted_objects[obj_name] = {str(k): v for k, v in r_obj.items()}
                    except AttributeError:
                        # If items() not available, store as is
                        converted_objects[obj_name] = r_obj
                else:
                    # Try general conversion
                    converted_objects[obj_name] = pandas2ri.rpy2py(r_obj)
                
            except Exception as e:
                print(f"Note: Could not convert object {obj_name}: {str(e)}")
                # Store the R object directly as fallback
                converted_objects[obj_name] = r_obj
    
    return converted_objects

In [49]:
rdata_path = "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_SET/Multiome_SETBP1/Multiome_SETBP1/all/AGG_Setbp1/outs/Multiome.RData"
data = load_rdata(rdata_path)

# Access converted objects
for obj_name, obj in data.items():
    print(f"Loaded object: {obj_name}, Type: {type(obj)}")

Stored Seurat object pbmc_RNA in original R format
Stored Seurat object pbmc_RNA_prova in original R format
Stored Seurat object pbmc_umap_coord in original R format
Loaded object: A, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: A2, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: all.genes, Type: <class 'list'>
Loaded object: Cluster, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: cluster.markers, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: clusters_coupe, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: DGE_embryo_ctrl_mut, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: DGE_P2_ctrl_mut, Type: <class 'pandas.core.frame.DataFrame'>
Loaded object: i, Type: <class 'numpy.ndarray'>
Loaded object: LogFC.onlypos, Type: <class 'numpy.ndarray'>
Loaded object: min.diff.pct, Type: <class 'numpy.ndarray'>
Loaded object: min.pct, Type: <class 'numpy.ndarray'>
Loaded object: mycol, Type: <class 'list'>
Loaded object: N, Type:

In [50]:
data.keys()

dict_keys(['A', 'A2', 'all.genes', 'Cluster', 'cluster.markers', 'clusters_coupe', 'DGE_embryo_ctrl_mut', 'DGE_P2_ctrl_mut', 'i', 'LogFC.onlypos', 'min.diff.pct', 'min.pct', 'mycol', 'N', 'p1', 'p2', 'p3', 'p5', 'pbmc_RNA', 'pbmc_RNA_prova', 'pbmc_umap_coord', 'PCA', 'test.use', 'thresh.use', 'top10'])

In [51]:
for key in data.keys():
    print(type(data[key]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'rpy2.robjects.methods.RS4'>
<class 'rpy2.robjects.methods.RS4'>
<class 'rpy2.robjects.methods.RS4'>
<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


In [55]:
for key in data.keys():
    print(f"\nKey: {key}")
    if type(data[key]) == pd.DataFrame:
        print("DataFrame:")
        print(data[key].head())
    elif type(data[key]) == np.ndarray:
        print("NumPy Array:")
        print(data[key][:10])
    elif type(data[key]) == list:
        print("List:")
        print(data[key][:10] if len(data[key]) > 10 else data[key])
    elif type(data[key]) == dict:
        print("Dictionary:")
        # Print first 5 key-value pairs
        for i, (k, v) in enumerate(data[key].items()):
            if i >= 5:
                break
            print(f"{k}: {v}")
    elif str(type(data[key])).find('RS4') != -1:
        print("R S4 Object")
        # print(str(data[key])[:200] + "..." if len(str(data[key])) > 200 else str(data[key]))
    else:
        print("Other type" )
        # print(f"Other type ({type(data[key])}:")
        # print(str(data[key])[:100])

R[write to console]: Error in print(..., self = self) : unused argument (useS4 = FALSE)



/home/kubacki.michal/.conda/envs/jupyter_nb/lib/python3.11/site-packages/rpy2/robjects/robject.py:123: UserWarning: Invalid call to "show()" in R: Error in print(..., self = self) : unused argument (useS4 = FALSE)

  warnings.warn(f'Invalid call to "show()" in R: {rre}')
R[write to console]: Error in print(..., self = self) : unused argument (useS4 = FALSE)

/home/kubacki.michal/.conda/envs/jupyter_nb/lib/python3.11/site-packages/rpy2/robjects/robject.py:123: UserWarning: Invalid call to "show()" in R: Error in print(..., self = self) : unused argument (useS4 = FALSE)

  warnings.warn(f'Invalid call to "show()" in R: {rre}')
R[write to console]: Error in print(..., self = self) : unused argument (useS4 = FALSE)




Key: A
DataFrame:
          Var1   Freq
1  embryo_ctrl   1438
2   embryo_mut   5765
3      P2_ctrl  12276
4       P2_mut  11925

Key: A2
DataFrame:
  Var1         Var2  Freq
1    0  embryo_ctrl    12
2    1  embryo_ctrl    26
3    2  embryo_ctrl   392
4    3  embryo_ctrl     1
5    4  embryo_ctrl    30

Key: all.genes
List:
['Xkr4', 'Gm1992', 'Gm19938', 'Gm37381', 'Rp1', 'Sox17', 'Gm37587', 'Gm37323', 'Mrpl15', 'Lypla1']

Key: Cluster
DataFrame:
                   orig.ident  nCount_RNA  nFeature_RNA Condition  \
AAACAGCCAACTGGGA-4        RNA      3573.0          1887    P2_mut   
AAACAGCCAAGGATTA-3        RNA      7486.0          3147   P2_ctrl   
AAACAGCCAAGTTATC-3        RNA      5815.0          2856   P2_ctrl   
AAACAGCCAATTGAGA-4        RNA      3482.0          1804    P2_mut   
AAACAGCCACCTCAGG-4        RNA      3919.0          1999    P2_mut   

                   RNA_snn_res.0.5  id             Barcode  
AAACAGCCAACTGGGA-4               0   0  AAACAGCCAACTGGGA-4  
AAACAGCCAAGG